# Hypothesis Testing

- Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

- Do some movie genres earn more revenue than others?

- Are some genres higher rated than others?


## Load Imports

In [1]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')

import warnings
warnings.filterwarnings('ignore')

plt.style.use(['seaborn-muted','fivethirtyeight'])
sns.set_context('talk')


In [2]:
# Install tmdbsimple package
!pip install tmdbsimple

## API Credentials 

In [3]:
# Loading saved API credentials
import json
with open('/Users/admin/.secret/imdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api_token', 'api-key'])

In [4]:
# Save login credentials
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

## Function Definitions

In [5]:
def get_movie_with_rating(movie_id):
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #loop through countries in releases
    for c in releases['countries']:
        #if the country abbrev = US
        if c['iso_3166_1'] == 'US':
            #save a certification key in info with the certification
            info['certification'] = c['certification']
    return info

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [7]:
#defining folder path and listing the files currently in Data folder
FOLDER = "Data/HYPOTHESIS TESTING - Years"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

[]

## Load Data

In [8]:
# Load basics data
basics = pd.read_csv("Data/basics_compressed_cleaned.csv.gz")
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
81877,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
81878,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
81879,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
81880,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [9]:
# Load akas data
akas = pd.read_csv("Data/akas_compressed_cleaned.csv.gz")
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
...,...,...,...,...,...,...,...,...
1452559,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0.0
1452560,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0.0
1452561,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0.0
1452562,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0.0


In [10]:
# Load ratings data
ratings = pd.read_csv("Data/ratings_compressed_cleaned.csv.gz")
ratings

,tconst,averageRating,numVotes
0,tt0035423,6.4,87440
1,tt0062336,6.4,179
2,tt0068865,5.4,74
3,tt0069049,6.7,7810
4,tt0088751,5.2,339
...,...,...,...
68051,tt9914942,6.6,181
68052,tt9915872,6.4,9
68053,tt9916170,7.0,7
68054,tt9916190,3.7,243


## Loop for API Calls

In [11]:
# Make API calls for years 2000 and 2001
years_to_get = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019] 

In [12]:
# Create an emplty list to store errors
errors = [ ]

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(years_to_get, desc='YEARS', position=0):
   
    # Defining the JSON file to store results for year
    JSON_file = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_file)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_file,'w') as f:
            json.dump([{'imdb_id':0}],f)


    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()



    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_file)


    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
        #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_file)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
            
    # Save the results
    final_year_df = pd.read_json(JSON_file)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

    
# Print return of number of movie_ids that caused error
print(f"- Total errors: {len(errors)}")


## Combine the Data for Years 2010 - 2019

## Export to csv.gz

In [15]:
# Save current unfiltered dataframe to file
merged_df.to_csv("Data/tmdb_results_combined_final.csv.gz",compression='gzip',index=False)

NameError: name 'merged_df' is not defined